# TruBot testing

This notebook tests a conversation bot with vector-store context of TruEra website. 

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [6]:
from trulens_eval.keys import *
from trulens_eval.examples.trubot import get_or_make_chain, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.util import instrumented_classes, jsonify
from trulens_eval.util import Obj, ObjSerial, Method, FunctionOrMethod, all_objects, Class
import json
import pydantic

from pprint import PrettyPrinter
pp = PrettyPrinter()

Tru().reset_database()

huggingface api: 0requests [00:00, ?requests/s]

In [7]:
chain = get_or_make_chain(cid=None)

'Starting a new conversation with 0/default.'
✅ chain 0/default -> default.sqlite
✅ feedback def. feedback_definition_hash_1682b60b001dddb30b8fd7a89c0d01d6 -> default.sqlite
✅ feedback def. feedback_definition_hash_5719fa2b309e711b5341552068b65e94 -> default.sqlite
✅ feedback def. feedback_definition_hash_4c88ee11aabb4808086d192aa4681202 -> default.sqlite


In [8]:
# Enumerate instrumented components:

from trulens_eval.utils.langchain import Is

tests = [Is.chain, Is.vector_store, Is.llm, Is.prompt, Is.retriever, Is.memory, Is.chathistory]

for q, ci, obj in instrumented_classes(jsonify(chain, instrument=chain.instrument)):

    assert isinstance(ci, Class), f"class_info was not a Class object, was {ci}, {type(ci)}."

    print(q, list(Is.what(ci)))

* []
*.model ['chain']
*.model.memory ['memory']
*.model.memory.llm ['llm']
*.model.memory.prompt ['prompt']
*.model.memory.chat_memory ['chathistory']
*.model.combine_docs_chain ['chain']
*.model.combine_docs_chain.llm_chain ['chain']
*.model.combine_docs_chain.llm_chain.prompt ['prompt']
*.model.combine_docs_chain.llm_chain.llm ['llm']
*.model.combine_docs_chain.document_prompt ['prompt']
*.model.question_generator ['chain']
*.model.question_generator.prompt ['prompt']
*.model.question_generator.llm ['llm']
*.model.retriever ['retriever']


In [9]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

Force stopping dashboard ...
Starting dashboard ...
Looking for things to do. Stop me with `tru.stop_evaluator()`.

In [11]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

selectors = selectors[2:3]
messages = messages[2:3]

def test_bot(selector, question):
    print(selector, question)
    chain = get_or_make_chain(cid=question + str(selector), selector=selector)
    answer = get_answer(chain=chain, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

TP().finish()

[autoreload of trulens_eval.schema failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    ref.__class__ = new
  Fil

3 Kim jest Shayak?


Looking for things to do. Stop me with `tru.stop_evaluator()`.

TypeError: _post_record() takes 6 positional arguments but 8 were given

Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.